In [1]:
from flask import Flask, render_template, request,jsonify
# from flask_cors import CORS,cross_origin
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pymongo
dbConn = pymongo.MongoClient("mongodb://localhost:27017/")  # opening a connection to Mongo
db = dbConn['crawlerDB'] # connecting to the database called crawlerDB
table = db['ABP']

for x in range(1,45):
    abp_url="https://www.anandabazar.com/topic/coronavirus?page="+str(x)#string and int can concatenate only when we use str before int
    uclient=uReq(abp_url)
    abpPage=uclient.read()
    uclient.close()
    abp_html=bs(abpPage,"html.parser")
    bigboxes=abp_html.find_all('div',{'class':"col-lg-6 col-xs-12"})
    try:     
        for box in bigboxes:
            news_link= "https:"+box.div.a['href']
            news_res=requests.get(news_link)
            news_html=bs(news_res.text,"html.parser")
            news_article=news_html.find('div',{'class':"col-12 abp-storypage-articlebody abp-videoarticle-content"})
            final_news=news_article.p.text
            output=news_article.find_all('p')
            var=""
            for out in output:
                var=var+out.text
            mydict={"news":var}
            table.insert_one(mydict)
    except:
        continue